In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sort_alpha import sorted_alpha
import random
import unidip.dip as dip
from sklearn.mixture import GaussianMixture
from scipy.spatial import distance

In [ ]:
def torso_size(data):
    neck = [data['x'].loc[1], data['y'].loc[1]]
    if neck ==0:
        return([0])
    hip1 = [data['x'].loc[8], data['y'].loc[8]]
    hip2 = [data['x'].loc[11], data['y'].loc[11]]
    if hip1[0]>0 and hip2[0]>0:
        probabilities = [data['p'].loc[8], data['p'].loc[11]]
        max_p = np.argmax(probabilities)
        if max_p==0:
            torso = distance.euclidean(hip1, neck)
        else:
            torso = distance.euclidean(hip2, neck)
    elif hip1[0]==0:
        torso = distance.euclidean(hip2, neck)
    else:
        torso = distance.euclidean(hip1, neck)
    return(torso)

In [ ]:
path_folders = Path('/Users/andreibirladeanu/Documents/Data/bounding_boxes')
folders = sorted_alpha(path_folders)


In [ ]:
path_openpose = Path('/Users/andreibirladeanu/Documents/Data/meal_tracked_bbox')

In [ ]:
openpose = sorted_alpha(path_openpose)


In [ ]:
openpose

In [ ]:
images = []

for folder,pose in zip(folders[0:1],openpose[0:1]):
    print(folder)
    folder_path = Path(os.path.join(path_folders, folder))
    list_images = []
    skeletons = []
    for skel in sorted_alpha(os.path.join(path_folders,folder)):
        skeletons.append(skel)
        if len(skeletons)>1:
            continue
        for bbox in sorted_alpha(Path(os.path.join(folder_path,skeletons[0]))):
            bboxes = sorted_alpha(Path(os.path.join(folder_path,sorted_alpha(folder_path)[0])))
            image = cv2.imread(os.path.join(folder_path,sorted_alpha(folder_path)[0],bbox))
            
            torso = pd.read_csv(os.path.join(path_openpose, pose, skel, bbox.split(".")[0]+'.csv'))
            mean = np.mean(image,axis=2, dtype=np.float32)
            list_images.append(np.mean([np.mean(mean),torso_size(torso)]))
    images.append(list_images)                          

In [ ]:
print('success')

In [ ]:
path_boxes = '/Users/andreibirladeanu/Documents/Data/bounding_boxes/1043_meal/skel1'
boxes = [x for x in sorted_alpha(path_boxes)]
boxes = [cv2.imread(path_boxes + x) for x in boxes]

In [ ]:
boxes = [np.mean(box,dtype=np.float32, axis=2) for box in boxes]
for_hist =  np.array([[np.mean(x)] for x in boxes])
for_dip =  np.array([np.mean(x) for x in boxes])

In [ ]:
### check 1053 and 1069 i.e. images[4] against 1132 [images[47]]

In [ ]:
plt.hist(images[4])

In [ ]:
plt.hist(images[-1])

In [ ]:
for_hist =  np.array([[x] for x in images[-2]])
gm = GaussianMixture(n_components=2, random_state=0).fit(for_hist)

In [ ]:
labels = gm.predict(for_hist) 
labels

In [ ]:
gm.means_

In [ ]:
plt.hist(images[-2])

In [ ]:
len(np.where(labels==1)[0]), len(np.where(labels==0)[0])

In [ ]:
data = np.msort(images[4])
print(dip.diptst(data))

In [ ]:
data = np.msort(images[8])
print(dip.diptst(data))

In [ ]:
dip = UniDip()

In [ ]:
for folder, histogram in zip(folders, images):
    if histogram == []:
        continue
    data =  np.msort(histogram)
    p_value = dip.diptst(data)[1]
    if p_value<0.001:
        print(p_value, folder)

In [ ]:
folders[19]

In [ ]:
images[19]

In [ ]:
edges.shape

In [ ]:
## perform KS test on 2 histograms

In [ ]:
distr1_path = '/Users/andreibirladeanu/Documents/Data/bounding_boxes/1148_meal/skel2/'
distr1 = [x for x in sorted_alpha(distr1_path)]

distr2_path = '/Users/andreibirladeanu/Documents/Data/bounding_boxes/1148_meal/skel3/'
distr2 = [x for x in sorted_alpha(distr2_path)]

In [ ]:
boxes1 = [np.mean(cv2.imread(distr1_path + x), axis=2, dtype=np.float32) for x in distr1]
boxes2 = [np.mean(cv2.imread(distr2_path + x), axis=2, dtype=np.float32) for x in distr2]

In [ ]:
for_hist1 = np.array([np.mean(x)for x in boxes1])
for_hist2 =np.array([np.mean(x)for x in boxes2])

In [ ]:
for_hist1

In [ ]:
from scipy import stats
stats.kstest(for_hist1, for_hist2)

In [ ]:
plt.hist(for_hist1), plt.hist(for_hist2)